<a href="https://colab.research.google.com/github/RohanSaxena14/mask_vs_no-mask/blob/master/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://github.com/RohanSaxena14/mask_vs_no-mask/raw/master/data/data_cropped.rar

In [0]:
!pip install patool

import patoolib

patoolib.extract_archive("data_cropped.rar", outdir="/content/")

In [8]:
import os
import cv2
import random
from google.colab.patches import cv2_imshow

DIR = "/content/data_cropped/"
data = []

label = 1

for folder_name in os.listdir(DIR):
  print(folder_name)
  for file_name in os.listdir(DIR + folder_name):
    image = cv2.imread(DIR + folder_name + "/" + file_name)
    image = cv2.resize(image, (300, 300))
    data.append([image, label])
  label = 0

random.shuffle(data)

without_mask
with_mask


In [0]:
import numpy as np

images_tensor = []
labels = []


for image, label in data:
  images_tensor.append(image)
  labels.append(label)

images_tensor = np.array(images_tensor).reshape(-1, 300, 300, 3)
labels = np.array(labels)

In [17]:
base_model = tf.keras.applications.vgg16.VGG16(include_top = False, input_shape = (300,300,3))

58892288/58889256 [==============================] - 1s 0us/step


In [18]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

In [0]:
for layer in base_model.layers[:17]:
  layer.trainable = False

In [0]:
import tensorflow as tf

model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(1024, activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(512, activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(256, activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [26]:
model.fit(images_tensor, labels, epochs = 5, batch_size = 32, validation_split = 0.2)

Epoch 1/5
30/30 [==============================] - 16s 524ms/step - loss: 0.9318 - accuracy: 0.9623 - val_loss: 0.3490 - val_accuracy: 0.9958
Epoch 2/5
30/30 [==============================] - 11s 352ms/step - loss: 0.9448 - accuracy: 0.9885 - val_loss: 0.5467 - val_accuracy: 0.9958
Epoch 3/5
30/30 [==============================] - 11s 353ms/step - loss: 0.6080 - accuracy: 0.9948 - val_loss: 0.5891 - val_accuracy: 0.9958
Epoch 4/5
30/30 [==============================] - 11s 351ms/step - loss: 0.7713 - accuracy: 0.9927 - val_loss: 0.4036 - val_accuracy: 0.9958
Epoch 5/5
30/30 [==============================] - 11s 352ms/step - loss: 3.6229 - accuracy: 0.9948 - val_loss: 1.5079 - val_accuracy: 0.9958


In [0]:
model.save("face_mask_model")

In [0]:
!zip -r /content/face_mask_model.zip /content/face_mask_model